# Benchmark of well-mixed stochastic solvers for a Schlogl model

STEPS simulation of the Schlogl model described in appendix A.4 of https://link.springer.com/978-3-319-63113-4, comparing the performance of the 'direct' and 'rssa' stochastic solvers for this model.

## Model

The model consists of three species, $\{A,B,X\}$, and four reactions implementing a trimolecular, autocatalytic reaction
scheme. The population of species A and B are highly abundant and assumed to be constant values (buffer molecules). The state of the system is thus represented by
the number of species X.

This are the four reactions, each with a reaction rate $c_i$:
\begin{align}
    A + 2X &{\overset{c_1}{\rightarrow}} 3X \\
    3X &{\overset{c_2}{\rightarrow}} A + 2X \\
    B &{\overset{c_5}{\rightarrow}} X \\
    X &{\overset{c_6}{\rightarrow}} B \\
\end{align}

The initial condition of species is #X = 250, #A = 1.0e5 and #B = 2.0e5. The
rate constants of reactions are $c_1$ = 3.0e–7, $c_2$ = 1.0e–4, $c_3$ = 1.0e–3 and $c_4$ = 3.5.


## Setup
We have reaction rates $c_i$ in units of $s^{-1}$, need to convert these to units of $M^{(1-n)}s^{-1}$ for the parameter `kcst` in STEPS, where $n$ is the order of the reaction, i.e. the number of molecules on the LHS.

The k is calculated diferently depending on the type of the reaction, as shown in the "2.3: Reaction propensity with mass action kinetics" of https://link.springer.com/978-3-319-63113-4

In [ ]:
# convert stochastic reaction rate in specified volume to STEPS kcst
# order is the number of molecules on the left hand side of the reaction
# maxSameMol is the maximum number of molecules of the same type on the left of the reaction
def get_kcst(stoch_rate, order, maxSameMol, volume):
    import numpy as np

    A = 6.02214076e23  # Avogadro number: molecules in 1 mol
    litres = 1e3 * volume
    if maxSameMol == 1:
        k = stoch_rate / np.power(A * litres, 1 - order)
    elif maxSameMol == 2:
        k = stoch_rate / np.power(A * litres, 1 - order) / 2
    elif maxSameMol == 3:
        k = stoch_rate / np.power(A * litres, 1 - order) / 6
    return k

In [ ]:
import numpy

import steps.model as smodel

mdl = smodel.Model()
A = smodel.Spec("A", mdl)
B = smodel.Spec("B", mdl)
X = smodel.Spec("X", mdl)
vsys = smodel.Volsys("vsys", mdl)

volume = 1.6667e-21

# Number of molecules
Na = 1.0e5
Nb = 2.0e5
Nx = 250

# stochastic reaction rate
c1 = 3.0e-7
c2 = 1.0e-4
c3 = 1.0e-3
c4 = 3.5

# Reactions
R1 = smodel.Reac(
    "R1", vsys, lhs=[A, X, X], rhs=[X, X, X], kcst=get_kcst(c1, 3, 2, volume)
)
R2 = smodel.Reac(
    "R2", vsys, lhs=[X, X, X], rhs=[A, X, X], kcst=get_kcst(c2, 3, 3, volume)
)
R3 = smodel.Reac("R3", vsys, lhs=[B], rhs=[X], kcst=get_kcst(c3, 1, 1, volume))
R4 = smodel.Reac("R4", vsys, lhs=[X], rhs=[B], kcst=get_kcst(c4, 1, 1, volume))

In [ ]:
import steps.geom as swm

wmgeom = swm.Geom()

comp = swm.Comp("comp", wmgeom)
comp.addVolsys("vsys")
comp.setVol(volume)

## Simulation

In [ ]:
import matplotlib.pyplot as plt
import steps.rng as srng
import steps.solver as ssolver

rng = srng.create("mt19937", 256)
rng.initialize(23412)

solver_direct = ssolver.Wmdirect(mdl, wmgeom, rng)
solver_rssa = ssolver.Wmrssa(mdl, wmgeom, rng)

timeAxis = numpy.arange(0.0, 50.001, 0.001)


def simulate(solver, niter=1, initX=Nx):
    solver.reset()
    solver.setCompCount("comp", "A", Na)
    solver.setCompCount("comp", "B", Nb)
    solver.setCompCount("comp", "X", Nx)

    import numpy

    # Allocate array for 50 sec
    res = numpy.zeros([niter, 50001, 1])
    # Posible X values
    histogram = numpy.zeros([1000])

    for i in range(0, niter):
        solver.reset()
        solver.setCompCount("comp", "A", Na)
        solver.setCompCount("comp", "B", Nb)
        solver.setCompCount("comp", "X", Nx)

        # A and B are constants
        solver.setCompClamped("comp", "A", True)
        solver.setCompClamped("comp", "B", True)
        for t in range(0, 50001):
            solver.run(timeAxis[t])
            numberX = solver.getCompCount("comp", "X")
            res[i, t, 0] = numberX
            histogram[int(numberX)] += 1

    return (res, histogram)


def plotres(res, res_=[]):
    # Plot number of molecules of 'molA' over the time range:
    plt.plot(timeAxis, res[:, 0], label="X")
    if len(res_) > 0:
        plt.plot(timeAxis, res_[:, 0], label="X'")

    plt.xlabel("Time (sec)")
    plt.ylabel("#molecules")
    plt.legend()
    plt.show()


def plothistogram(histogram, histogram_=[]):
    plt.plot(range(1000), histogram, label="appearances")
    if len(histogram_) > 0:
        plt.plot(range(1000), histogram_, label="appearances'")

    plt.xlabel("#molecules")
    plt.ylabel("appearances")
    plt.legend()
    plt.show()

## Bistability
The Schlogl model is a remarkable example of a reaction network which exhibits bistability.
Most of the time just changing the initial population of X by one (250 and 249) we will be able to see 2 stable trends, one in blue around 600 and another on red around 100.

### Direct

In [ ]:
res, histogram = simulate(solver_direct)
res_, histogram_ = simulate(solver_direct, initX=249)

res_mean = numpy.mean(res, 0)
res_mean_ = numpy.mean(res_, 0)

plotres(res_mean, res_mean_)
plothistogram(histogram, histogram_)

### RSSA

In [ ]:
res, histogram = simulate(solver_rssa)
res_, histogram_ = simulate(solver_rssa, initX=249)

res_mean = numpy.mean(res, 0)
res_mean_ = numpy.mean(res_, 0)

plotres(res_mean, res_mean_)
plothistogram(histogram, histogram_)

## Benchmark
Finally we check the execution time of both direct and rssa solvers. Both tend to an average of around 300 X molecules. As we saw before, this is due to the bistability of the model.

Overal rssa performs slightly better.

In [ ]:
import time

start = time.time()
resDirect, histogram = simulate(solver_direct, niter=100)
end = time.time()
timeDirect = end - start

start = time.time()
resRSSA, histogram = simulate(solver_rssa, niter=100)
end = time.time()
timeRSSA = end - start

res_mean_Direct = numpy.mean(resDirect, 0)
res_mean_RSSA = numpy.mean(resRSSA, 0)
plotres(res_mean_Direct, res_mean_RSSA)

print("Time on Direct solver (s)", timeDirect)
print("Time on RSSA solver (s)", timeRSSA)

In [ ]:
x = numpy.arange(2)
plt.bar(x, height=[timeDirect, timeRSSA])
plt.xticks(x, ["Direct", "RSSA"])

plt.xlabel("Solvers")
plt.ylabel("time(s)")
plt.show()